# Python ETL Script for GDP Data

## Introduction

Hello, I'm João Henrique. In this project, we'll demonstrate a practical application of Python for an ETL (Extract, Transform, Load) process. Our focus is on extracting Global GDP data, transforming it, and loading it into both a CSV file and a MySQL database. This showcases Python's utility in web scraping, data processing, and database operations.

## Project Overview

- **Data Extraction**: 
  - Extract GDP data of countries from a Wikipedia page using web scraping techniques.
- **Data Transformation**: 
  - Process and convert the GDP data into a more usable format.
- **Data Load**: 
  - Load the transformed data into a CSV file and a MySQL database.
- **Query Data**: 
  - Perform SQL queries to retrieve and analyze the data.

## Technical Aspects

- **Libraries Used**: 
  - `requests`, `pandas`, `BeautifulSoup`, `lxml`, `numpy`, `mysql.connector`.
- **Data Source**: 
  - Wikipedia page on countries by GDP (nominal).

## Execution Guide

- **Setting Up**: 
  - Ensure all required Python libraries are installed.
  - Set up MySQL database credentials and parameters as per your environment.

- **Running the Code**: 
  - The script is divided into functions, each handling a part of the ETL process.
  - Run the entire script to see the ETL process or step through each function individually.

- **Logging**: 
  - Logs are written to `log_file.txt`, useful for debugging or tracking the process.

## Code Structure

- **Extract Function**: 
  - Scrapes the webpage and extracts the GDP data.
- **Transform Function**: 
  - Transforms the data into a more analysis-friendly format.
- **Load to CSV Function**: 
  - Saves the data into a CSV file.
- **Load to Database Function**: 
  - Inserts the data into a MySQL database.
- **Query and Logging Functions**: 
  - Facilitates querying the database and logging the process.



## Project Requirements

To successfully run this notebook, the following software and libraries are required:

### Software
- **Python (version 3.11.3)**: The Python programming language, essential for running the code.
- **MySQL (version 8.0.34)**: The database workbench used for storing and managing data.

### Python Libraries
- **pandas (version 2.0.3)**: A powerful data manipulation and analysis library.
- **requests (version 2.31.0)**: Used for making HTTP requests, essential for web scraping.
- **mysql.connector (version 8.2.0)**: Enables connection and interaction with MySQL databases.
- **numpy (np)**: Adds support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays.
- **datetime**: Provides classes for manipulating dates and times.
- **BeautifulSoup**: A library for pulling data out of HTML and XML files, used for web scraping.
- **Numpy**: A library for handling mathematical functions with precision and speed

These libraries can be installed using pip:
```bash
pip install requests pandas numpy mysql-connector-python beautifulsoup4


In [13]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
from datetime import datetime
import mysql.connector


In [14]:
URL = "https://web.archive.org/web/20230902185326/https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29"
CSV_PATH = 'Countries_by_GDP.csv'
SQL_CONNECTION = ['127.0.0.1', 'root', '112104', '6666']
LOG_PATH = 'log_file.text'
TABLE_ATTRIBS = ['Country','GDP_USD_millions']
DB_NAME = 'World_Economies'
TABLE_NAME = 'Countries_by_GDP'


## Function Definitions

Each part of the ETL process is encapsulated in a function. This makes the code easier to understand and maintain.


## Function Description for `log_progress`

### Purpose
Logs a message with a timestamp to a specified log file.

### Process
1. **Timestamp Generation**: Generates a timestamp in the specified format.
2. **Writing to Log File**: Appends the message along with the timestamp to the log file.

### Parameters
- `message`: The message to be logged.

In [15]:
def log_progress(message):
    timestamp_format = "%Y-%h-%d-%H:%M:%S"
    now = datetime.now()
    timestamp = now.strftime(timestamp_format)
    with open(LOG_PATH,"a") as f:
        f.write(timestamp+","+message+"\n")

- Just a function to show the log in the end of this documment

In [16]:
def open_log():
    f = open(LOG_PATH, 'r')
    file_contents = f.read()
    print (file_contents)
    f.close()

# Extracting Data from HTML Tables

This function is designed to parse and extract data from HTML tables on a specified webpage.

## Function Description

### Extracting Data from HTML Tables:

1. **Fetching the Webpage**: The function begins by making a GET request to the URL provided in the `url` argument. The HTML content of the webpage is then loaded into a `BeautifulSoup` object for parsing.

2. **Parsing the HTML**: The function locates all table bodies (`<tbody>`) in the HTML content. Specifically, it focuses on the third table body, as indexed by `tables[2]`.

3. **Extracting Table Data**: It iterates over all table rows (`<tr>`) within the targeted table. For each row, it finds all data cells (`<td>`) and checks if they meet certain conditions (e.g., not empty, contains a hyperlink, and doesn't have a specific character). If conditions are met, it extracts the relevant data (country name and GDP in millions of USD) and adds it to a dataframe.

4. **Creating and Updating the DataFrame**: A new dataframe is created for each row of data and then appended to the main dataframe `df`. This process continues for each row in the HTML table.

5. **Returning the DataFrame**: After iterating through all the rows, the function returns the final dataframe containing the extracted data.

### Parameters:
- `url`: The URL of the webpage from which to extract the table data.
- `table_attribs`: A list of column names for the dataframe.

### Returns:
- A pandas DataFrame containing the extracted data from the specified HTML table.



In [17]:
def extract(url, table_attribs):
    webpage = requests.get(url).text
    soup = BeautifulSoup(webpage,'html.parser')
    
    df = pd.DataFrame(columns=table_attribs)
    
    tables = soup.find_all('tbody')
    rows = tables[2].find_all('tr')

    for row in rows :
        col = row.find_all('td')
        if len(col)!=0:
            if col[0].find('a') is not None and '—' not in col[2]:
                data_dict = {"Country": col[0].a.contents[0],
                             "GDP_USD_millions": col[2].contents[0]}
                df1 = pd.DataFrame(data_dict, index=[0])
                df = pd.concat([df,df1], ignore_index=True)
    print(df)
    return df

In [18]:
try:
    df=extract(URL,TABLE_ATTRIBS)
    log_progress("Data extracted successfuly")
except Exception as e:
    log_progress(f"Error while extracting data: {e}")


              Country GDP_USD_millions
0       United States       26,854,599
1               China       19,373,586
2               Japan        4,409,738
3             Germany        4,308,854
4               India        3,736,882
..                ...              ...
186  Marshall Islands              291
187             Palau              262
188          Kiribati              248
189             Nauru              151
190            Tuvalu               65

[191 rows x 2 columns]


## Function Description for `transform`

### Purpose
The `transform` function is responsible for converting GDP values from millions to billions and preparing the data for further analysis or storage.

### Process
1. **Converting Data Type**: The function starts by extracting the 'GDP_USD_millions' column from the dataframe and converting the values from string to float. This involves removing commas from the numerical strings and casting them to float.

2. **Scaling the Values**: The GDP values, initially in millions, are then divided by 1,000 to convert them into billions. This scaling is rounded to two decimal places for precision and readability.

3. **Updating the DataFrame**: Finally, the function updates the dataframe by replacing the original 'GDP_USD_millions' column with the newly scaled values and renaming the column to 'GDP_USD_billions' to reflect the change in units.

### Returns
- A pandas DataFrame with the updated GDP values in billions.

In [19]:
def transform(df):
    GDP_list = df["GDP_USD_millions"].tolist()
    GDP_list = [float("".join(x.split(','))) for x in GDP_list]
    GDP_list = [np.round(x/1000,2) for x in GDP_list]
    df["GDP_USD_millions"] = GDP_list
    df = df.rename(columns={"GDP_USD_millions":"GDP_USD_billions"})
    print(df)
    return df

In [20]:
try:       
    df=transform(df)
    log_progress("Data transformed successfuly")
except Exception as e:
    log_progress(f"Error while transforming data: {e}")

              Country  GDP_USD_billions
0       United States          26854.60
1               China          19373.59
2               Japan           4409.74
3             Germany           4308.85
4               India           3736.88
..                ...               ...
186  Marshall Islands              0.29
187             Palau              0.26
188          Kiribati              0.25
189             Nauru              0.15
190            Tuvalu              0.06

[191 rows x 2 columns]


## Function Description for `load_to_csv`

### Purpose
The `load_to_csv` function is designed to save the processed dataframe to a CSV file, allowing for easy storage and accessibility of the data.

### Process
1. **Exporting to CSV**: The function takes the dataframe and the path where the CSV file should be saved. It then uses pandas' `to_csv` method to write the dataframe into a CSV file at the specified location.

### Parameters
- `df`: The dataframe to be saved.
- `csv_path`: The file path where the CSV file will be stored.

### Returns
- This function does not return a value; its primary purpose is to save the dataframe as a CSV file.


In [21]:
def load_to_csv(df):
    df.to_csv(CSV_PATH)
    print(f"data loaded to:{CSV_PATH}")

In [22]:
try:       
    load_to_csv(df)
    log_progress("Data loaded to CSV successfuly")
except Exception as e:
    log_progress(f"Error while loading data to CSV: {e}") 

data loaded to:Countries_by_GDP.csv


## Function Description for `connect_database`

### Purpose
The `connect_database` function establishes a connection to a MySQL database using provided connection parameters.

### Process
1. **Establishing Connection**: Utilizes the `mysql.connector.connect` method to create a connection to the MySQL database. The connection parameters (host, user, password, and port) are specified in the `sql_connection` list.

### Parameters
- `sql_connection`: A list containing connection parameters (host, user, password, port).

### Returns
- A connection object to the MySQL database.

In [23]:
def connect_database():
    
    mydb = mysql.connector.connect(
        
    host=SQL_CONNECTION[0],
    user=SQL_CONNECTION[1],
    password=SQL_CONNECTION[2],
    port=SQL_CONNECTION[3]
    )   
    print(mydb)
    return mydb

In [24]:
try: 
    connect_database()
    log_progress("Database successfuly connected")
except Exception as e:
    log_progress(f"Error while connecting to DB: {e}")   

## Function Description for `load_to_db`

### Purpose
The `load_to_db` function is responsible for creating a database schema and table, and then loading data from a pandas DataFrame into the MySQL database table.

### Process
1. **Database Connection**: Connects to the MySQL database using the `connect_database` function.
2. **Schema and Table Creation**: Creates a new schema and table in the database if they do not exist. The table structure is defined based on the dataframe's columns.
3. **Data Insertion**: Converts the dataframe into records and inserts each record into the database table using an INSERT statement.

### Parameters
- `df`: The pandas DataFrame to be loaded into the database.

In [27]:
def load_to_db(df):
    db = connect_database()
    cursor = db.cursor()
    print("Database connection established.")
            
    column_names = df.columns.tolist()
    data_records = df.to_records(index=False)

    # Create the schema and table
    cursor.execute(f"CREATE SCHEMA IF NOT EXISTS `{DB_NAME}`")
    cursor.execute(f"USE `{DB_NAME}`")
    print(f"Schema `{DB_NAME}` accessed/created.")
    
    columns_sql = ', '.join([f"`{col}` VARCHAR(255)" if df[col].dtype == 'object' else f"`{col}` FLOAT" for col in column_names])
    cursor.execute(f"CREATE TABLE IF NOT EXISTS `{TABLE_NAME}` (id INT AUTO_INCREMENT PRIMARY KEY, {columns_sql})")
    print(f"Table `{TABLE_NAME}` accessed/created.")
            
    # Prepare the INSERT statement
    placeholders = ', '.join(['%s'] * len(column_names))
    DML = f"INSERT INTO `{TABLE_NAME}` ({', '.join([f'`{col}`' for col in column_names])}) VALUES ({placeholders})"

    # Insert each record
    for record in data_records:
        values = tuple(record[col] for col in column_names)
        cursor.execute(DML, values)

    db.commit()
    db.close()
    print("Data inserted successfully into the database.")

In [28]:
try:       
    load_to_db(df)
    log_progress("Data loaded to DB successfuly")
except Exception as e:
    log_progress(f"Error while loading data to DB: {e}")   


Database connection established.
Schema `World_Economies` accessed/created.
Table `Countries_by_GDP` accessed/created.
Data inserted successfully into the database.


## Function Description for `run_query`

### Purpose
Executes a specified SQL query on the connected MySQL database and prints the query result.

### Process
1. **Database Connection**: Connects to the MySQL database using the `connect_database` function.
2. **Executing Query**: Executes the provided SQL query and retrieves the results using pandas' `read_sql` function.

### Parameters
- `query_statement`: The SQL query to be executed.

### Returns
- The executed query statement.

In [29]:
def run_query(query_statement):
    db = connect_database()
    cursor = db.cursor()
    cursor.execute(f"Use {DB_NAME}")
    print(pd.read_sql(query_statement,db))
    return query_statement

# Make the query yourself!

You can know more about querys in my <a href="https://balenciagaa.notion.site/SQL-1b15be8f10d74695a8de7bc860292f1b?pvs=73">SQL tutorial</a> (It's in portuguese right now but i will fix soon): 


## Query examples:

**Retrieve Records with Specific Conditions**

`SELECT * FROM Countries_by_GDP WHERE GDP_USD_billions > 50 ORDER BY GDP_USD_billions DESC`

**Calculate Average GPD**

`SELECT AVG(GDP_USD_billions) AS average_GDP FROM Countries_by_GDP`

**Pick the top 5 GDP**

`SELECT Country FROM Countries_by_GDP ORDER BY GDP_USD_billions DESC LIMIT 5`

In [30]:
try:       
    quey_statement = run_query(f"SELECT * FROM Countries_by_GDP")
except Exception as e:
    log_progress(f"Error while querying data: {e}")
finally:
        log_progress(f"Query made: {quey_statement}")

        id           Country  GDP_USD_billions
0        1     United States          26854.60
1        2             China          19373.60
2        3             Japan           4409.74
3        4           Germany           4308.85
4        5             India           3736.88
...    ...               ...               ...
1332  1333  Marshall Islands              0.29
1333  1334             Palau              0.26
1334  1335          Kiribati              0.25
1335  1336             Nauru              0.15
1336  1337            Tuvalu              0.06

[1337 rows x 3 columns]


C:\Users\joaoh\AppData\Local\Temp\ipykernel_17848\1288848911.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  print(pd.read_sql(query_statement,db))


- Show the log progess of the code

In [31]:
open_log()

2023-Dec-21-22:07:22,Data extracted successfuly
2023-Dec-21-22:07:27,Data transformed successfuly
2023-Dec-21-22:08:50,Data loaded to CSV successfuly
2023-Dec-21-22:09:17,Database successfuly connected
2023-Dec-21-22:12:15,Data loaded to DB successfuly
2023-Dec-21-22:12:51,Query made: SELECT * FROM Countries_by_GDP



## Final Considerations

### Summary of the Project
In this ETL project, we successfully developed and executed a pipeline to extract, transform, and load data regarding global GDP figures. The process involved retrieving data from a web source, manipulating the data for consistency and scale, and ultimately storing it in a MySQL database for further analysis and retrieval.

### Key Learnings
- **Python's Adaptability in Data Extraction**: This project showcased Python's strength in web scraping and data extraction, demonstrating its ability to interact with web content and parse HTML.
- **Complex Data Transformation**: We highlighted the importance and methods of transforming raw data into a more analyzable format, particularly converting GDP figures from millions to billions.
- **Database Interaction and Data Storage**: The project provided practical experience in loading data into a MySQL database, emphasizing the significance of data storage in structured formats for efficient retrieval and analysis.
- **Dynamic Data Handling**: Addressing various challenges, such as dynamic data structures in web tables and ensuring seamless database connectivity, was a crucial learning aspect.

### Conclusion
This ETL project served as an excellent practical application of data extraction, transformation, and loading using Python. It allowed us to delve deep into aspects of data handling, from web scraping to database management, underlining the vital role of ETL processes in making data useful for analysis and decision-making.

I hope this project has provided valuable insights into ETL processes, demonstrating the versatility of Python in handling and manipulating data for meaningful outcomes. Thank you for engaging with this project, and I look forward to any future collaborations or discussions on similar data-driven initiatives.

- Special thanks to IBM, without the awesome Data Engeneering Course this project would be more difficult to do ;)